In [1]:
using LinearAlgebra, RowEchelon, LaTeXStrings, Plots, SymPy
include("LAcodes.jl");  ##] dev --local "."

LAcodes.title( "The QR Decomposition", sz=30, color="darkred")

HTML{String}("<div><p style=\"color:darkred;font-size:30pt;height:15px;text-align:left;\">The QR Decomposition</p></div>")

# 1. Gram-Schmidt and the QR Decomposition

## 1.1 Gram-Schmidt and QR

##### **The Gram-Schmidt Equations**

<div style="float:left;width:55%;background-color:#F2F5A9;">

**The Gram-Schmidt Procedure**
$$
\begin{align}
w_1 & = \color{red}{v_1} && & \quad q_1 = \frac{1}{\lVert w_1 \rVert} w_1 \\
w_2 & = \color{red}{v_2} - Proj^\perp_{ span\{ w_1      \} } \; \color{red}{v_2} &=&\ \color{red}{v_2} - \frac{\color{red}{v_2} \cdot w_1}{w_1 \cdot w_1} w_1& \quad q_2 = \frac{1}{\lVert w_2 \rVert} w_2 \\ 
w_3 & = \color{red}{v_3} - Proj^\perp_{ span\{ w_1, w_2 \} } \; \color{red}{v_3} & =&\ \color{red}{v_3} - \frac{\color{red}{v_3} \cdot w_1}{w_1 \cdot w_1} w_1
                                                                 - \frac{\color{red}{v_3} \cdot w_2}{w_2 \cdot w_2} w_2 & \quad q_3 = \frac{1}{\lVert w_3 \rVert} w_3 \\   
 \dots& \dots\dots\dots\dots\dots\dots & &\dots\dots\dots\dots\dots\dots\dots & \\
w_k & = \color{red}{v_k} - Proj^\perp_{ span\{ w_1, w_2, \dots w_{k-1} \} } \; \color{red}{v_k} &&
& \quad q_k = \frac{1}{\lVert w_k \rVert} w_k 
\end{align}
$$
</div>

<div style="float:left;width:40%;">

**Remark:**
Scaling $w$ does not affect the projection terms:
$$
\begin{align}
& \tilde{w} & =&\;\; \color{red}{\alpha}\; w \\
& \frac{w \cdot v}{w \cdot w} w &=&\; \frac{\color{red}{\alpha}\; w \cdot v}{\color{red}{\alpha^2}\; w \cdot w} \;\color{red}\alpha\; w 
\end{align}
$$
</div>
<div style="float:right;width:40%;border:1px solid black;padding:10pt">

We can replace the $w_i$ in the Gram-Schmidt equations by $q_i$ for example.
</div>

##### **The QR Decomposition**

<div style="float:left;width:55%;">

Let us rewrite these equations: solve for $v_i$ in terms of the $q_i$:

$$
\begin{align}
\color{red}{v_1} && = &\; \lVert w_1 \rVert\; \color{red}{q_1}  \\
\color{red}{v_2} & = w_2 + Proj^\perp_{ span\{ w_1      \} } \; {v_2} &=&\ \frac{{v_2} \cdot q_1}{q_1 \cdot q_1}\; \color{red}{q_1} + \lVert w_2 \rVert\; \color{red}{q_2} \\ 
\color{red}{v_3} & = w_3 + Proj^\perp_{ span\{ w_1, w_2 \} } \;  {v_3} & =&\ \frac{{v_3} \cdot q_1}{w_1 \cdot q_1}\; \color{red}{q_1}
                                                                 + \frac{{v_3} \cdot q_2}{q_2 \cdot q_2}\; \color{red}{q_2} + \lVert w_3 \rVert\; \color{red}{q_3}  \\   
 \dots& \dots\dots\dots\dots\dots\dots & &\dots\dots\dots\dots\dots\dots\dots & \\
\end{align}
$$
</div>
<div style="float:right;width:35%;height:4.6cm;background-color:#F2F5A9;">

$\quad$ We see  $\color{red}{A = Q R}, \quad $ where

$$
\begin{align}
A &= \begin{pmatrix} v_1 & v_2 & \dots & v_{\color{red}{n}} \end{pmatrix}\\
Q &= \begin{pmatrix} q_1 & q_2 & \dots q_{\color{red}k} \end{pmatrix} \\
R &= \begin{pmatrix} \lVert w_1 \rVert & \frac{{v_2} \cdot q_1}{q_1 \cdot q_1} & \dots \\
                        0 & \lVert w_2 \rVert & \dots \\
                        \dots & \dots & \dots
\end{pmatrix}
\end{align}
$$
</div>

<div style="float:left;width:55%;height:3.8cm;background-color:#F2F5A9;">

**Theorem:** Let $A$ be a matrix in $\mathbb{R}^{M \times N}$.<br>
$\quad\quad$ Then $A$ has a factorization $A = Q R$ such that
* $Q^t Q = I$
* $R$ is an upper triangular matrix
* If the columns of $A$ are linearly independent, $R$ is invertible
</div>
<div style="float:right;width:35%;">

**Remark:**
* Gram-Schmidt constructs these matrices.
* The off diagonal entries of $R$ consist of the coefficients  computed for the projections

$$
A = Q R \Leftarrow \color{red}{R = Q^t A}
$$
</div>

## 1.2 QR and the Normal Equation

<div style="float:left;">
We had arrived at the normal equation by multiplying $$A x = b - b_\perp$$
$\quad$ with $A^t$ to remove the $b_\perp$ term.

We only need any one solution $x$.<br>
$\quad\quad$ As we did before, assume that $A$ is full-column rank,<br>
$\quad\quad$ (i.e., remove any linearly dependent colums of the original matrix problem)
</div>
<img style="float:center;border:1px solid black;" width=200 src="OrthoProjection_into_plane.png">

----

<div style="float:left;width:55%;height:4.2cm;padding:5pt;background-color:#F2F5A9;">

**Theorem:** Let $A$ in $\mathbb{R}^{M \times N}$ with reduced QR factorization $A = Q R$.<br>
    $\quad\quad$ Then $ A^t A x = A^t b \Leftrightarrow R x = Q^t b $

<br>

**Remark:** This theorem state that we can use the $Q R$ decomposition<br>
    $\quad\quad$ to solve the normal equation.
</div>
<div style="float:right;width:35%;">
Assume $A$ has full column rank, so $A = Q R$,<br> where $R$ (and hence $R^t$) is invertible:
$A = Q R \Leftrightarrow A^t = R^t Q^t$

$$\begin{align}
A x = b  &\Rightarrow A^t A x = A^t b \\
         & \Leftrightarrow R^t Q^t Q R x = R^t Q^t b \\
         & \Leftrightarrow Q^t Q R x = Q^t b \\
         & \Leftrightarrow R x = Q^t b
\end{align}$$
<div>

## 1.3 Important Properties of Matrices with Orthonormal Columns

Let $Q$ have orthonormal columns:<br>
$\quad\quad$ Multiplying vectors by $Q$ **does not change the length** of a vector<br>
$\quad\quad$ and **does not change the angle** between vectors

> This is an attractive property for numerical computations:<br>
> **Computations involving orthogonal vectors will not overflow!**

<div style="float:left;width:55%;height:3.4cm;padding:5pt;background-color:#F2F5A9;">

**Theorem:** Let $Q$ be an orthogonal (or a unitary) matrix, and let $x$ and $y$ be vectors of consistent length
* $\lVert Q x \rVert =  \lVert x \rVert, \quad\quad\quad\;$ i.e., **lengths are conserved**
* $(Q x) \cdot (Q y) = x \cdot y,\quad$ i.e., **angles are conserved**
*  $Q x \perp Q y \Leftrightarrow x \perp y, \quad$ i.e. **orthgonal anlges are conserved.**
</div>
<div style="float:right;width:35%;">

Let $Q$ have orthonormal columns, i.e., $Q^t Q = I$

$$\begin{align}
    \left( (Q x) \cdot (Q y) \right) &= (Q x)^t (Q y) \\
    &= x^t Q^t Q y \\
    &= \left( x \cdot y \right) 
\end{align}$$
</div>

# 2. Computing the QR factorization

**Remark:**
The Gram Schmidt equations as written have **poor numerical properties.**
> Practical Implementations introduce some **highly successful** modifications
> * Immediately remove $q_i$ components from the remaining $v_i$ vectors
> * Reorder the remaining $v_i$ vectors at each step

**Naive Method:**